# Transfer learning

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize, Compose, Resize

batch_size = 64
# Define the necessary transformations
preprocess = Compose([
    ToTensor(),
    Resize((300, 300)),
    Resize((300, 300)),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageFolder(root="data/horse-or-human", transform=preprocess)
test_dataset = ImageFolder(root="data/validation-horse-or-human", transform=preprocess)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [2]:
# dispositivo de entrenamiento
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Usando: {device}")

Usando: cuda


In [3]:
# bucle de entrenamiento

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # modo entrenamiento
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # mover si es necesario
        X = X.to(device)
        y = y.to(device)
        # forward
        # prediccion y costo
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        
        loss, current = loss.item(), batch * batch_size + len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# funcion de pruebas
def test_loop(dataloader, model, loss_fn):
    # modo evaluacion
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    # evaluacion del modelo con torch.no_grad
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Usando un modelo Pre-entrenado
En este caso usaremos el modelo VGG16 pre entrenado en el dataset de IMAGENET

In [4]:
from torchvision import models

class CustomVgg(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.backbone = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        self.backbone.classifier[6] = nn.Linear(in_features=4096, out_features=2)

    def forward(self, x):
        output = self.backbone(x)
        return output
    
model = CustomVgg().to(device)
print(model)

/home/pepe/miniconda3/envs/cv_unifranz/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pepe/miniconda3/envs/cv_unifranz/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomVgg(
  (backbone): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [5]:
learning_rate = 1e-4
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# bucle principal
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.722797  [   64/ 1027]
loss: 0.350002  [  128/ 1027]
loss: 0.067040  [  192/ 1027]
loss: 0.007250  [  256/ 1027]
loss: 0.003532  [  320/ 1027]
loss: 0.139581  [  384/ 1027]
loss: 0.041451  [  448/ 1027]
loss: 0.207897  [  512/ 1027]
loss: 0.000014  [  576/ 1027]
loss: 0.051922  [  640/ 1027]
loss: 0.000000  [  704/ 1027]
loss: 0.000086  [  768/ 1027]
loss: 0.155786  [  832/ 1027]
loss: 0.000055  [  896/ 1027]
loss: 0.000000  [  960/ 1027]
loss: 0.000298  [ 1024/ 1027]
loss: 0.000000  [ 1027/ 1027]


/home/pepe/miniconda3/envs/cv_unifranz/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Test Error: 
 Accuracy: 94.9%, Avg loss: 0.234415 

Epoch 2
-------------------------------
loss: 0.000004  [   64/ 1027]
loss: 0.000010  [  128/ 1027]
loss: 0.000005  [  192/ 1027]
loss: 0.047826  [  256/ 1027]
loss: 0.000001  [  320/ 1027]
loss: 0.000027  [  384/ 1027]
loss: 0.000000  [  448/ 1027]
loss: 0.018385  [  512/ 1027]
loss: 0.000001  [  576/ 1027]
loss: 0.000000  [  640/ 1027]
loss: 0.000021  [  704/ 1027]
loss: 0.000008  [  768/ 1027]
loss: 0.000175  [  832/ 1027]
loss: 0.000405  [  896/ 1027]
loss: 0.030651  [  960/ 1027]
loss: 0.009570  [ 1024/ 1027]
loss: 0.000004  [ 1027/ 1027]
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.049653 

Epoch 3
-------------------------------
loss: 0.000000  [   64/ 1027]
loss: 0.000000  [  128/ 1027]
loss: 0.000000  [  192/ 1027]
loss: 0.000000  [  256/ 1027]
loss: 0.000000  [  320/ 1027]
loss: 0.000000  [  384/ 1027]
loss: 0.000788  [  448/ 1027]
loss: 0.000030  [  512/ 1027]
loss: 0.000000  [  576/ 1027]
loss: 0.000000  [  640/ 1027]
loss: 

# Fine tuning

In [6]:
class CustomVgg(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.backbone = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        for param in self.backbone.parameters():
                param.requires_grad = False
        self.backbone.classifier[6] = nn.Linear(in_features=4096, out_features=2)

    def forward(self, x):
        output = self.backbone(x)
        return output

In [7]:
learning_rate = 1e-4
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# bucle principal
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.000000  [   64/ 1027]
loss: 0.000000  [  128/ 1027]
loss: 0.000000  [  192/ 1027]
loss: 0.000000  [  256/ 1027]
loss: 0.000000  [  320/ 1027]
loss: 0.000000  [  384/ 1027]
loss: 0.000000  [  448/ 1027]
loss: 0.000000  [  512/ 1027]
loss: 0.000000  [  576/ 1027]
loss: 0.000000  [  640/ 1027]
loss: 0.000000  [  704/ 1027]
loss: 0.000000  [  768/ 1027]
loss: 0.000000  [  832/ 1027]
loss: 0.000000  [  896/ 1027]
loss: 0.000000  [  960/ 1027]
loss: 0.000000  [ 1024/ 1027]
loss: 0.000000  [ 1027/ 1027]
Test Error: 
 Accuracy: 99.6%, Avg loss: 0.005083 

Epoch 2
-------------------------------
loss: 0.000000  [   64/ 1027]
loss: 0.000000  [  128/ 1027]
loss: 0.000000  [  192/ 1027]
loss: 0.000000  [  256/ 1027]
loss: 0.000000  [  320/ 1027]
loss: 0.000000  [  384/ 1027]
loss: 0.000000  [  448/ 1027]
loss: 0.000000  [  512/ 1027]
loss: 0.000000  [  576/ 1027]
loss: 0.000000  [  640/ 1027]
loss: 0.000000  [  704/ 1027]
loss: 0.000000  [  768/ 1027